This genetic algorithm code here below was created using the example in the lecture, but instead of using the y function is used the dericate of the y function (y_prima) as the fitness function

You can play with the number of generations (generation_max) and the crossover probability (Cp)

In [189]:
import numpy as np
import random

In [225]:
class GeneticA:
  #Class that includes all necessary variables and methods for the genetic algorithm 
  generation = 0 #We initialise the generation in zero
  generation_max =5 #We expect to get maximum other 2 generations to solve this problem
  cp=0.7 #Crossover probability between 0.6 and 0.8
  mp=0.001 #Mutation probability
  num_bits=7 #number of bits in each string
  num_individuals_pop = 4
  population = None
  population_bin= None
  fitness = None
  mating_pool = None
  change_to_binary = None
  change_to_decimal = None
  new_population = None
  
  
  def __init__(self):
    #initialise population
    num = 0
    vals=[]
    while (num < self.num_individuals_pop):
      vals.append(random.randint(0,25)) #Any number from 0 to 25
      num+=1
    #vals = vals.sort()
    self.population= np.array(vals)
    print("Generated values", self.population)
    self.change_to_binary = np.vectorize(self.chg_to_bin)
    self.change_to_decimal = np.vectorize(self.chg_to_dec)
    self.population_bin = self.change_to_binary(self.population)
    print("Values in binary", self.population_bin)
    print("Testing conversion in decimal", self.change_to_decimal(self.population_bin))
  
  def chg_to_bin(self, val):
    #Covert any decimal to a binary string
    bin_num = str()
    first= True
    while (val!=0):
      if(first):
        bin_num= bin_num + str(val%2)
      else:
        bin_num= str(val%2)+bin_num
      first=False
      val=val//2
          #ensuring there are always 5 bits able to represent numbers 
    #frpm 0 to 31 (32 values)
    while(len(bin_num)<self.num_bits):
      bin_num="0"+bin_num

    return bin_num


  def chg_to_dec(self, val_strg):
    #Convert any binary string into a decimal number
    dec_num = 0
    num = len(val_strg)-1
    for i in val_strg:
      dec_num = dec_num + (int(i)*(2**num))
      num-=1
    return dec_num

  def fun(self, x):
     #Function to optimise and that will be used as fitness function
     #y= 15+ 8*x-(x**2)
     #Instead of using the function like in the class,
     #We use the first derivate, since if the first derivate is 
     #equals to zero it can be a local maxima or local minima
     y_prima = 8-2*x
     return y_prima

  def relative_fitness(self):
    funct = np.vectorize(self.fun)
    fit= funct(self.population)
    print("Fitness of each individual", fit)
    #Getting the absolute value of the fitness
    abs_fit = lambda x:abs(x)
    print("Absolute Fitness of each individual", abs_fit(fit))
    #Total of all the fitness of the individuals
    total = np.sum(abs_fit(fit))
    print("total sum", total)
    def rel_fit(fit_val):
      return fit_val/total
    rela_fit = np.vectorize(rel_fit)
    self.fitness = rela_fit(abs_fit(fit))
    print("The relative fitness of each invidual is:", self.fitness )

  def selection(self):
    self.mating_pool= np.array([])
    for i in range(len(self.fitness)):
      #We want to minimise the error and we select all the values
      # that have a relative fitness less or euqls than 0.25 
      if (round(self.fitness[i],2)<=0.25):
          print("val", self.population[i], "with fitness", self.fitness[i])
          self.mating_pool = np.append(self.mating_pool, self.population_bin[i])
    
    print("The mating pool is: " , self.mating_pool)
    print("Size of maiting pool", np.shape(self.mating_pool)[0])
    if(np.shape(self.mating_pool)[0]<1):
      #If nobody was selected. I use the population that I have for crossover 
      self.mating_pool = self._population
    elif(np.shape(self.mating_pool)[0]==1):
      #I replicate the individual 3 more times in the mating pool
      self.mating_pool = np.append(self.mating_pool, self.mating_pool[0])
      self.mating_pool = np.append(self.mating_pool, self.mating_pool[0])
      self.mating_pool = np.append(self.mating_pool, self.mating_pool[0])
    elif(np.shape(self.mating_pool)[0]==2):
      #I copy the two individuals that I have again to have 4 individuals
      self.mating_pool = np.append(self.mating_pool,self.mating_pool[0])
      self.mating_pool = np.append(self.mating_pool,self.mating_pool[1])
    elif(np.shape(self.mating_pool)[0]==3):
      #I copy individual one again
      self.mating_pool = np.append(self.mating_pool,self.mating_pool[0])
    else:
      pass
      #I do not do nothing, I have my set complete
    print("The mating pool is: " , self.mating_pool)
    print("Size of maiting pool", np.shape(self.mating_pool)[0])
    


  def crossover(self):
    num = self.num_individuals_pop
    new_population = np.array([])
    second_array=[1,2,3,0]
    j=0
    for i in range (num):
      val_ran = random.random()
      if(val_ran>self.cp):
        print("xor", self.mating_pool[i], self.mating_pool[j])
        offspring = self.change_to_decimal(self.mating_pool[i]) ^ self.change_to_decimal(self.mating_pool[j])
        print("offspring", offspring)  
        new_population = np.append(new_population, self.change_to_binary(offspring))
      else:
        new_population = np.append(new_population, self.mating_pool[i])
      j+=1
    self.new_population = new_population
    print("The new population after crossover is:", self.new_population)
        
  def mutation(self):
    for i in range(self.num_individuals_pop):
      val_ran = random.random()
      int_index = random.randint(0,4)
      aux = ""
      if(val_ran<self.mp):
        print("before mutation",self.new_population[i])
        aux = list(self.new_population[i])
        bit = aux[int_index]
        if(bit=="1"):
          aux[int_index] = "0"
        else:
          aux[int_index] = "1"
        aux2 =""
        for s in aux:
          aux2=aux2+s
        print("after mutation",aux2)
        self.new_population[i] = aux2
      
    print("new population after mutation",self.new_population)

   
    


In [226]:
#Initialising the genetic algorithms
g = GeneticA()
max_gen = 0
while (max_gen<g.generation_max):
  print("___________________This is the iteration", max_gen, "______________")
  g.relative_fitness()
  g.selection()
  g.crossover()
  g.mutation()
  #Updating the population for the new generation
  g.population_bin = g.new_population
  print("New population in the main", g.population_bin)
  g.population =g.change_to_decimal(g.new_population)
  max_gen+=1

print(g.population)
g.relative_fitness()

Generated values [13  3 25 19]
Values in binary ['0001101' '0000011' '0011001' '0010011']
Testing conversion in decimal [13  3 25 19]
___________________This is the iteration 0 ______________
Fitness of each individual [-18   2 -42 -30]
Absolute Fitness of each individual [18  2 42 30]
total sum 92
The relative fitness of each invidual is: [0.19565217 0.02173913 0.45652174 0.32608696]
val 13 with fitness 0.1956521739130435
val 3 with fitness 0.021739130434782608
The mating pool is:  ['0001101' '0000011']
Size of maiting pool 2
The mating pool is:  ['0001101' '0000011' '0001101' '0000011']
Size of maiting pool 4
xor 0001101 0001101
offspring 0
xor 0001101 0001101
offspring 0
xor 0000011 0000011
offspring 0
The new population after crossover is: ['0000000' '0000011' '0000000' '0000000']
new population after mutation ['0000000' '0000011' '0000000' '0000000']
New population in the main ['0000000' '0000011' '0000000' '0000000']
___________________This is the iteration 1 ______________
Fitne

It was expected 4, but as you can see the final selected value is in the final population and the error will be the value that has the smallest fitness minus the expected value. So in case that the value is 5 and the expected was 4, the error is: 4-5= -1 or if the value is 3 the error is 4-3 = 1